In [1]:
import os
import tensorflow as tf

use_cpu = False

if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    os.environ['CPU_ONLY'] = "TRUE"

    physical_devices = tf.config.list_physical_devices('CPU')

    tf.config.set_logical_device_configuration(
        physical_devices[0],
        [tf.config.LogicalDeviceConfiguration() for i in range(8)])
    logical_devices = tf.config.list_logical_devices('CPU')

    print(logical_devices)
else:
    os.environ['CPU_ONLY'] = "FALSE"
    physical_devices = tf.config.list_physical_devices('GPU')
    print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from latentneural.data import DataManager
from latentneural import TNDM
from latentneural.runtime import Runtime, ModelType
from latentneural.utils import AdaptiveWeights


data_dir = os.path.join('..', '..', 'latentneural', 'data', 'storage', 'lorenz', '20210610T215300')
dataset, settings = DataManager.load_dataset(
    directory=data_dir,
    filename='dataset.h5')

In [3]:
print(settings)
print('\nDataset keys:', dataset.keys())

{'step': 0.01, 'stop': 1, 'neurons': 30, 'base_rate': 5, 'latent_dim': 3, 'relevant_dim': 2, 'behaviour_dim': 4, 'conditions': 1, 'trials': 100, 'initial_conditions': '<function uniform.<locals>.callable at 0x7f92b42ac940>', 'selected_condition': 0, 'seed': 12345, 'train_pct': 0.7, 'valid_pct': 0.1, 'test_pct': 0.2, 'created': '2021-06-10T21:56:46.415999'}

Dataset keys: dict_keys(['behaviour_weights', 'neural_weights', 'relevant_dims', 'test_behaviours', 'test_behaviours_noiseless', 'test_data', 'test_latent', 'test_rates', 'time_data', 'train_behaviours', 'train_behaviours_noiseless', 'train_data', 'train_latent', 'train_rates', 'valid_behaviours', 'valid_behaviours_noiseless', 'valid_data', 'valid_latent', 'valid_rates'])


In [4]:
neural_data = dataset['train_data'].astype('float')
valid_neural_data = dataset['valid_data'].astype('float')
behavioural_data = dataset['train_behaviours'].astype('float')
valid_behavioural_data = dataset['valid_behaviours'].astype('float')

In [ ]:
from datetime import datetime
from collections import defaultdict
logdir = os.path.join(data_dir, 'results', 'tndm', datetime.now().strftime("%Y%m%d-%H%M%S"))

optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-2,
    beta_1=0.9, 
    beta_2=0.999,
    epsilon=1e-01)

layers_settings=defaultdict(lambda: dict(
    kernel_initializer=tf.keras.initializers.VarianceScaling(
        scale=1.0, mode='fan_in', distribution='normal'),
    kernel_regularizer=tf.keras.regularizers.l2(l=0.1)
))
layers_settings['encoder'].update(dict(dropout=0.05, var_min=0.1, var_max=0.1))
layers_settings['relevant_decoder'].update(dict(kernel_regularizer=tf.keras.regularizers.l2(l=1),
                                      recurrent_regularizer=tf.keras.regularizers.l2(l=1),
                                      original_cell=False))    
layers_settings['irrelevant_decoder'].update(dict(kernel_regularizer=tf.keras.regularizers.l2(l=1),
                                      recurrent_regularizer=tf.keras.regularizers.l2(l=1),
                                      original_cell=False))    
layers_settings['behavioural_dense'].update(dict(behaviour_type='synchronous',      
                                                behaviour_sigma=1))    
model = Runtime.train(
    model_type=ModelType.TNDM,
    adaptive_lr=dict(factor=0.95, patience=10, min_lr=1e-5),
    model_settings=dict(
        relevant_factors=2,
        irrelevant_factors=1,
        encoded_space=64,
        max_grad_norm=200,
        encoded_var_max=0.1,
        encoded_var_min=0.1,
        timestep=settings['step']
    ),
    layers_settings=layers_settings,
    optimizer=optimizer, 
    epochs=1000, 
    logdir=logdir,
    train_dataset=(neural_data, behavioural_data), 
    val_dataset=(valid_neural_data, valid_behavioural_data),
    adaptive_weights=AdaptiveWeights(
        initial=[1.0, 1.0, .0, .0, .0],
        update_start=[0, 0, 1000, 1000, 0],
        update_rate=[0., 0., 0.0005, 0.0005, 0.0005],
        min_weight=[1.0, 1.0, 0.0, 0.0, 0.0]
    ),
    batch_size=16
)

/home/alessandrofacchin/miniconda/envs/latentneural/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/1000
5/5 [==============================] - 3s 452ms/step - loss: 2935.6184 - loss/neural: 1785.2829 - loss/behavioural: 1230.3740 - loss/relevant_kldiv: 48.5635 - loss/irrelevant_kldiv: 48.4761 - loss/reg: 1936.7819 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.0000e+00 - weights/irrelevant_kldiv: 0.0000e+00 - weights/reg: 5.8333e-04 - learning_rate: 0.0100 - grads/EncoderGRUForward_grucell2: 19.1930 - grads/EncoderGRUBackward_grucell1: 18.9829 - grads/EncoderDense: 5088.5833 - grads/RelevantDenseMean: 21.7158 - grads/IrrelevantDenseMean: 13.0241 - grads/RelevantDecoderGRU_grucell3: 303.7006 - grads/IrrelevantDecoderGRU_grucell4: 203.3547 - grads/RelevantFactorsDense: 86.0689 - grads/IrrelevantFactorsDense: 45.6479 - grads/SynchronousBehaviouralDense: 7.1701 - grads/NeuralDense: 125.5040 - norms/EncoderGRUForward_grucell2: 1581.9101 - norms/EncoderGRUBackward_grucell1: 1576.2432 - norms/EncoderDense: 5950.7000 - norms/Releva

5/5 [==============================] - 1s 274ms/step - loss: 2195.9478 - loss/neural: 996.5209 - loss/behavioural: 1208.3715 - loss/relevant_kldiv: 51.0493 - loss/irrelevant_kldiv: 50.6485 - loss/reg: 1937.5610 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.0000e+00 - weights/irrelevant_kldiv: 0.0000e+00 - weights/reg: 0.0131 - learning_rate: 0.0100 - grads/EncoderGRUForward_grucell2: 263.5868 - grads/EncoderGRUBackward_grucell1: 273.2908 - grads/EncoderDense: 60970.8581 - grads/RelevantDenseMean: 412.8616 - grads/IrrelevantDenseMean: 218.1321 - grads/RelevantDecoderGRU_grucell3: 8750.4817 - grads/IrrelevantDecoderGRU_grucell4: 4198.2515 - grads/RelevantFactorsDense: 3097.0879 - grads/IrrelevantFactorsDense: 1027.6549 - grads/SynchronousBehaviouralDense: 421.6003 - grads/NeuralDense: 6793.7859 - norms/EncoderGRUForward_grucell2: 17950.4326 - norms/EncoderGRUBackward_grucell1: 17886.1903 - norms/EncoderDense: 67598.6315 - norms/Relevan

5/5 [==============================] - 1s 228ms/step - loss: 1956.9268 - loss/neural: 764.2478 - loss/behavioural: 1194.8969 - loss/relevant_kldiv: 61.4037 - loss/irrelevant_kldiv: 59.3671 - loss/reg: 1936.5994 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.0000e+00 - weights/irrelevant_kldiv: 0.0000e+00 - weights/reg: 0.0256 - learning_rate: 0.0100 - grads/EncoderGRUForward_grucell2: 640.6462 - grads/EncoderGRUBackward_grucell1: 663.1656 - grads/EncoderDense: 137649.9557 - grads/RelevantDenseMean: 1218.1860 - grads/IrrelevantDenseMean: 685.4124 - grads/RelevantDecoderGRU_grucell3: 14912.6932 - grads/IrrelevantDecoderGRU_grucell4: 7571.6103 - grads/RelevantFactorsDense: 5565.7467 - grads/IrrelevantFactorsDense: 2004.1370 - grads/SynchronousBehaviouralDense: 1099.3920 - grads/NeuralDense: 12985.2188 - norms/EncoderGRUForward_grucell2: 34321.4886 - norms/EncoderGRUBackward_grucell1: 34198.8558 - norms/EncoderDense: 129644.0638 - norms/R

5/5 [==============================] - 1s 232ms/step - loss: 1940.6237 - loss/neural: 746.5421 - loss/behavioural: 1175.4265 - loss/relevant_kldiv: 98.7129 - loss/irrelevant_kldiv: 91.4549 - loss/reg: 1928.0460 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.0000e+00 - weights/irrelevant_kldiv: 0.0000e+00 - weights/reg: 0.0381 - learning_rate: 0.0100 - grads/EncoderGRUForward_grucell2: 1044.5476 - grads/EncoderGRUBackward_grucell1: 1084.7439 - grads/EncoderDense: 209805.9609 - grads/RelevantDenseMean: 2488.0529 - grads/IrrelevantDenseMean: 1521.7728 - grads/RelevantDecoderGRU_grucell3: 22083.7116 - grads/IrrelevantDecoderGRU_grucell4: 11403.9109 - grads/RelevantFactorsDense: 9108.5990 - grads/IrrelevantFactorsDense: 3202.4703 - grads/SynchronousBehaviouralDense: 2059.5506 - grads/NeuralDense: 16473.4001 - norms/EncoderGRUForward_grucell2: 50697.7803 - norms/EncoderGRUBackward_grucell1: 50518.1790 - norms/EncoderDense: 192707.8958 - nor

...

5/5 [==============================] - 1s 265ms/step - loss: 1396.6824 - loss/neural: 708.8715 - loss/behavioural: 598.2314 - loss/relevant_kldiv: 88.9598 - loss/irrelevant_kldiv: 49.8182 - loss/reg: 1068.1761 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.5131 - weights/irrelevant_kldiv: 0.5131 - weights/reg: 1.0000 - learning_rate: 0.0023 - grads/EncoderGRUForward_grucell2: 727889.3229 - grads/EncoderGRUBackward_grucell1: 823999.2500 - grads/EncoderDense: 13292898.6667 - grads/RelevantDenseMean: 789618.6406 - grads/IrrelevantDenseMean: 91940.6257 - grads/RelevantDecoderGRU_grucell3: 3174657.2292 - grads/IrrelevantDecoderGRU_grucell4: 293657.1667 - grads/RelevantFactorsDense: 767361.1979 - grads/IrrelevantFactorsDense: 69607.3079 - grads/SynchronousBehaviouralDense: 25842.3831 - grads/NeuralDense: 124077.3086 - norms/EncoderGRUForward_grucell2: 1383430.0104 - norms/EncoderGRUBackward_grucell1: 1422475.6771 - norms/EncoderDense: 13439

5/5 [==============================] - 1s 216ms/step - loss: 1398.3947 - loss/neural: 699.6621 - loss/behavioural: 601.8017 - loss/relevant_kldiv: 89.4792 - loss/irrelevant_kldiv: 49.8236 - loss/reg: 1058.3935 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.5256 - weights/irrelevant_kldiv: 0.5256 - weights/reg: 1.0000 - learning_rate: 0.0023 - grads/EncoderGRUForward_grucell2: 737865.2292 - grads/EncoderGRUBackward_grucell1: 834895.8854 - grads/EncoderDense: 13452972.2500 - grads/RelevantDenseMean: 803766.9323 - grads/IrrelevantDenseMean: 93174.1491 - grads/RelevantDecoderGRU_grucell3: 3231304.9792 - grads/IrrelevantDecoderGRU_grucell4: 298673.0365 - grads/RelevantFactorsDense: 778167.0312 - grads/IrrelevantFactorsDense: 70626.3197 - grads/SynchronousBehaviouralDense: 25982.9787 - grads/NeuralDense: 125386.8802 - norms/EncoderGRUForward_grucell2: 1400014.5625 - norms/EncoderGRUBackward_grucell1: 1440053.3229 - norms/EncoderDense: 13620

5/5 [==============================] - 1s 223ms/step - loss: 1396.5221 - loss/neural: 705.9337 - loss/behavioural: 598.0390 - loss/relevant_kldiv: 88.0798 - loss/irrelevant_kldiv: 49.6467 - loss/reg: 1048.8077 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.5381 - weights/irrelevant_kldiv: 0.5381 - weights/reg: 1.0000 - learning_rate: 0.0021 - grads/EncoderGRUForward_grucell2: 746793.1615 - grads/EncoderGRUBackward_grucell1: 844800.1719 - grads/EncoderDense: 13599490.7500 - grads/RelevantDenseMean: 816768.2969 - grads/IrrelevantDenseMean: 94150.0156 - grads/RelevantDecoderGRU_grucell3: 3284292.3750 - grads/IrrelevantDecoderGRU_grucell4: 302948.4740 - grads/RelevantFactorsDense: 789582.7135 - grads/IrrelevantFactorsDense: 71504.0137 - grads/SynchronousBehaviouralDense: 26129.2025 - grads/NeuralDense: 126532.3333 - norms/EncoderGRUForward_grucell2: 1416577.5417 - norms/EncoderGRUBackward_grucell1: 1457606.1875 - norms/EncoderDense: 13801

5/5 [==============================] - 1s 212ms/step - loss: 1406.6263 - loss/neural: 703.2319 - loss/behavioural: 603.7655 - loss/relevant_kldiv: 89.4900 - loss/irrelevant_kldiv: 49.7238 - loss/reg: 1043.1427 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.5506 - weights/irrelevant_kldiv: 0.5506 - weights/reg: 1.0000 - learning_rate: 0.0021 - grads/EncoderGRUForward_grucell2: 757830.6562 - grads/EncoderGRUBackward_grucell1: 856837.9062 - grads/EncoderDense: 13772340.0833 - grads/RelevantDenseMean: 831390.7396 - grads/IrrelevantDenseMean: 95217.3275 - grads/RelevantDecoderGRU_grucell3: 3335694.0417 - grads/IrrelevantDecoderGRU_grucell4: 307519.8229 - grads/RelevantFactorsDense: 800312.6406 - grads/IrrelevantFactorsDense: 72429.8353 - grads/SynchronousBehaviouralDense: 26274.3986 - grads/NeuralDense: 127688.9167 - norms/EncoderGRUForward_grucell2: 1433115.4583 - norms/EncoderGRUBackward_grucell1: 1475132.6458 - norms/EncoderDense: 13981

5/5 [==============================] - 1s 222ms/step - loss: 1405.3118 - loss/neural: 697.6063 - loss/behavioural: 600.8177 - loss/relevant_kldiv: 86.4588 - loss/irrelevant_kldiv: 49.7434 - loss/reg: 1034.7204 - weights/neural_loglike: 1.0000 - weights/behavioural_loglike: 1.0000 - weights/relevant_kldiv: 0.5631 - weights/irrelevant_kldiv: 0.5631 - weights/reg: 1.0000 - learning_rate: 0.0020 - grads/EncoderGRUForward_grucell2: 768302.6146 - grads/EncoderGRUBackward_grucell1: 868369.4688 - grads/EncoderDense: 13937702.0000 - grads/RelevantDenseMean: 845876.6510 - grads/IrrelevantDenseMean: 96328.1022 - grads/RelevantDecoderGRU_grucell3: 3393762.6458 - grads/IrrelevantDecoderGRU_grucell4: 311891.2370 - grads/RelevantFactorsDense: 811019.0417 - grads/IrrelevantFactorsDense: 73246.9095 - grads/SynchronousBehaviouralDense: 26415.1523 - grads/NeuralDense: 128915.1914 - norms/EncoderGRUForward_grucell2: 1449636.3333 - norms/EncoderGRUBackward_grucell1: 1492643.2396 - norms/EncoderDense: 14160

In [ ]:
import latentneural.plot as lnp
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


log_f, b, _, _, _, _ = model(neural_data.astype('float'), training=False)
fig_rates = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=np.exp(log_f[1, :, :].numpy().T), colorscale="Blues"))

fig_rates.update_layout(title='Firing Rates of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_rates_n = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=dataset['train_rates'][1, :, :].T, colorscale="Blues"))

fig_rates_n.update_layout(title='Firing Rates of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_rates.show()
fig_rates_n.show()

In [ ]:
log_f, b, (g0, mean, logvar), _, (z_r, z_i), _ = model(neural_data.astype('float'), training=False)

fig_z = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=z_r.numpy()[0, :, :].T, colorscale="Blues"))

fig_z.update_layout(title='Latent Variables of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_z_n = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=dataset['train_latent'][0, :, :].T, colorscale="Blues"))

fig_z_n.update_layout(title='Latent Variables of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_z.show()
fig_z_n.show()

In [ ]:
latent_reconstructed_unsorted = np.concatenate([z_r.numpy().T, z_i.numpy().T], axis=0)
latent_original = dataset['train_latent'].T

print(latent_reconstructed_unsorted.shape)
print(latent_original.shape)

In [ ]:
latent_reconstructed_unsorted_r = latent_reconstructed_unsorted.reshape(
    latent_reconstructed_unsorted.shape[0], 
    latent_reconstructed_unsorted.shape[1] * latent_reconstructed_unsorted.shape[2]).T

latent_original_r = latent_original.reshape(
    latent_original.shape[0], 
    latent_original.shape[1] * latent_original.shape[2]).T

In [ ]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(latent_reconstructed_unsorted_r, latent_original_r)
print('Score: ', clf.score(latent_reconstructed_unsorted_r, latent_original_r))

predictions = clf.predict(latent_reconstructed_unsorted_r).T.reshape(latent_original.shape[0], latent_reconstructed_unsorted.shape[1], latent_reconstructed_unsorted.shape[2])

In [ ]:
fig = go.Figure()
condition=0
for d in range(3):
    # Create and style traces
    fig.add_trace(go.Scatter(x=dataset['time_data'], y=latent_original[d,:,condition], name='%d original' % (d),
                            line=dict(color=px.colors.qualitative.Plotly[d], width=10), opacity=0.2))
    fig.add_trace(go.Scatter(x=dataset['time_data'], y=predictions[d,:,condition], name='%d reconstructed' % (d),
                            line = dict(color=px.colors.qualitative.Plotly[d], width=2)))

# Edit the layout
fig.update_layout(title='Latent Trajectories of Condition #%d, Original vs Reconstructed' % (condition),
                xaxis_title='Time',
                yaxis_title='Value')
fig.show()

# Vaidation Performance

In [ ]:
log_f, b, (g0, mean, logvar), _, (z_r, z_i), _ = model(dataset['valid_data'].astype('float'), training=False)
latent_reconstructed_unsorted = np.concatenate([z_r.numpy().T, z_i.numpy().T], axis=0)
latent_original = dataset['valid_latent'].T
latent_reconstructed_unsorted_r = latent_reconstructed_unsorted.reshape(
    latent_reconstructed_unsorted.shape[0], latent_reconstructed_unsorted.shape[1] * latent_reconstructed_unsorted.shape[2]).T
latent_original_r = latent_original.reshape(
    latent_original.shape[0], latent_original.shape[1] * latent_original.shape[2]).T

clf = Ridge(alpha=1.0)
clf.fit(latent_reconstructed_unsorted_r, latent_original_r)
print('Score: ', clf.score(latent_reconstructed_unsorted_r, latent_original_r))

# Behaviour Reconstruction

In [ ]:
fig = go.Figure()
b_noiseless = dataset['valid_latent'][:,:,-dataset['behaviour_weights'].shape[0]:] @ \
    dataset['behaviour_weights'][:,:]
    
condition=0
for d in range(3):
    # Create and style traces
    fig.add_trace(go.Scatter(x=dataset['time_data'], y=b_noiseless[condition,:,d], name='%d original noisless' % (d),
                            line=dict(color=px.colors.qualitative.Plotly[d], width=10), opacity=0.2))
    fig.add_trace(go.Scatter(x=dataset['time_data'], y=b[condition,:,d], name='%d reconstructed' % (d),
                            line = dict(color=px.colors.qualitative.Plotly[d], width=2)))

# Edit the layout
fig.update_layout(title='Behavioural Trajectories of Condition #%d, Original vs Reconstructed' % (condition),
                xaxis_title='Time',
                yaxis_title='Value')
fig.show()